<a href="https://colab.research.google.com/github/Gilangp/Machine-learning/blob/main/week%207/TG1_JS07_2341720042_Gilang_Purnomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Praktikum 1**

In [ ]:
!pip install annoy
!pip install faiss-cpu
!pip install faiss-gpu
!pip install hnswlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551812 sha256=1acffe27cda0b12fc456857a6cabc95a1f4a29f9d19996dd2f791c55eeb4df8c
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
Successfully built annoy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 62.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528146 sha256=7877782e0b814f15d7773dd98b22db7beacae23491deada6d80ee3b87dbd8a5f
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/c

In [ ]:
import numpy as np
import pandas as pd
import time
from annoy import AnnoyIndex
import faiss
import hnswlib

def test_ann_algorithms(n_points, dim, n_queries=100):
    np.random.seed(42)
    data = np.random.random((n_points, dim)).astype('float32')
    queries = np.random.random((n_queries, dim)).astype('float32')

    results = {}

    # ============ ANNOY ============
    annoy_index = AnnoyIndex(dim, 'euclidean')
    start = time.time()
    for i in range(n_points):
        annoy_index.add_item(i, data[i])
    annoy_index.build(10)
    build_time = time.time() - start

    start = time.time()
    for q in queries:
        annoy_index.get_nns_by_vector(q, 5)
    query_time = time.time() - start

    results['ANNOY'] = (build_time, query_time)

    # ============ FAISS ============
    index_faiss = faiss.IndexFlatL2(dim)
    start = time.time()
    index_faiss.add(data)
    build_time = time.time() - start

    start = time.time()
    index_faiss.search(queries, 5)
    query_time = time.time() - start

    results['FAISS'] = (build_time, query_time)

    # ============ HNSW ============
    hnsw_index = hnswlib.Index(space='l2', dim=dim)
    start = time.time()
    hnsw_index.init_index(max_elements=n_points, ef_construction=100, M=16)
    hnsw_index.add_items(data)
    build_time = time.time() - start

    start = time.time()
    hnsw_index.knn_query(queries, k=5)
    query_time = time.time() - start

    results['HNSW'] = (build_time, query_time)

    return results


configs = [
    (1000, 2),
    (1000, 5),
    (1000000, 2),
    (1000000, 5)
]

records = []

for n_points, dim in configs:
    print(f"\nMengukur performa untuk {n_points} data / {dim}D ...")
    res = test_ann_algorithms(n_points, dim)
    for algo, (build, query) in res.items():
        records.append({
            'Jumlah Data/Dimensi': f'{n_points}/{dim}D',
            'Algoritma': algo,
            'Waktu Build (detik)': round(build, 4),
            'Waktu Query (detik)': round(query, 4)
        })

df = pd.DataFrame(records)
table = df.pivot(index='Jumlah Data/Dimensi', columns='Algoritma', values=['Waktu Build (detik)', 'Waktu Query (detik)'])

print("\n=== HASIL PERCOBAAN ===")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.expand_frame_repr', False)

print(table.to_string())


Mengukur performa untuk 1000 data / 2D ...

Mengukur performa untuk 1000 data / 5D ...

Mengukur performa untuk 1000000 data / 2D ...

Mengukur performa untuk 1000000 data / 5D ...

=== HASIL PERCOBAAN ===
                    Waktu Build (detik)                  Waktu Query (detik)                
Algoritma                  ANNOY         FAISS    HNSW          ANNOY         FAISS    HNSW 
Jumlah Data/Dimensi                                                                         
1000/2D                     0.0428       0.0001   0.0568        0.0023        0.0083  0.0007
1000/5D                     0.0279       0.0000   0.0678        0.0021        0.0088  0.0009
1000000/2D                 32.2413       0.0024  49.8331        0.0045        0.1258  0.0009
1000000/5D                 22.2578       0.0151  81.6183        0.0041        0.1416  0.0014
